# Resizing

## What interpolation method do you use?

In general, I find that using cv2.
INTER_AREA obtains the best results when resizing; however,
other appropriate choices include cv2.INTER_LINEAR,
cv2.INTER_CUBIC, and cv2.INTER_NEAREST

## Process

1. Find the aspect ratio by dividing the new width by the old width (which is the number of columns, so `.shape[1]`
2. Create an array for dimensions by using the new width and multiplying the height (which is the number of rows, so `.shape[0]` by the ratio
3. Pass in the image, dimensions, and interpolation parameters (usually `INTER_AREA` into the function
4. See the magic!

# Shifting an Image

## Process

1. Basically create the transformation matrix. One for the x, one for the y. X will look like `[1, 0, Tx]`, Y will look like `[0, 1, Ty]`. Negative values will shift to the left and up, positive values will shift to the right and down
2. Use the `warpAffine` function and pass in the width (columns) and height (rows)

In [1]:
import numpy as np
import argparse
import imutils
import cv2

ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True, help = "Path to the image")
args = vars(ap.parse_args())

image = cv2.imread(args["image"])
cv2.imshow("Original", image)
cv2.waitKey(0)

# First array is [1, 0, tx], which is the number of pixels to shift left or right
## Negative values shift image left and positive right
# Second array is [0, 1, ty], which is the number of pixels to shift up or down
## Negative values shift the image up and positive down
M = np.float32([[1, 0, 25], [0, 1, 50]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Down and Right", shifted)
cv2.waitKey(0)

M = np.float32([[1,0,-50],[0,1,-90]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Up and Left", shifted)
cv2.waitKey(0)

shifted = imutils.translate(image, 0, 100)
cv2.imshow("Shifted Down", shifted)
cv2.waitKey(0)

ModuleNotFoundError: No module named 'imutils'

# Rotating an Image

## Process

1. Calculate the center of the image (assumed to be where the people start rotating)
2. Call the `getRotationMatrix2D` to rotate the image a certain amount
3. Call the `warpAffine` function to pass in the image, rotation matrix, and dimensions

In [2]:
import argparse
import cv2
import numpy as np
import imutils

ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="Path to Image")
args = vars(ap.parse_args())

image = cv2.imread(args["image"])
cv2.imshow("Original Image", image)

# Grab the shape
(h, w) = image.shape[0:2]
# Calculate the center of the image
## Use half of the height and half of the width
center = (h // 2, w // 2)

# Pass in the center, angle, and scale of the image
M = cv2.getRotationMatrix2D(center, 45, 1.0)
# Output dimensions are in width and height
# We use (h, w) for the image because we say rows by columns, which is height by width
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by 45 Degrees", rotated)

# Pass in the center, angle, and scale of the image
M = cv2.getRotationMatrix2D(center, -90, 1.0)
# Output dimensions are in width and height
# We use (h, w) for the image because we say rows by columns, which is height by width
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by -90 Degrees", rotated)

rotated = imutils.rotate(image, 180)
cv2.imshow("Rotated by 180 Degrees", rotated)
cv2.waitKey(0)

ModuleNotFoundError: No module named 'cv2'

# Flipping Images

## Process

1. Just use the `.flip` function with the image, and what type of flip.

* `-1` for flipping both horizontally and vertically
* `1` for flipping horizontal because 1 would signify columns which if you flipped over a constant column would be horizontal
* `0` for flipping vertical because 0 signifies rows which if you flipped over a constant row would be vertical

# `imutils.py`

In [3]:
import numpy as np
import cv2

def translate(image, x, y):
    M = np.float32([[1, 0, x], [0, 1, y]])
    shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return shifted

def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def resize(image, width = None, height = None, inter=cv2.INTER_AREA):
    dimensions = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        # No resizing necessary
        return image

    if width is None:
        # We want to resize the height
        ratio = height / float(h)
        dimensions = (int(w * ratio), height)
    else:
        # We want to resize the width
        ratio = width / float(w)
        dimensions = (width, int(h * ratio))

    resized = cv2.resize(image, dimensions, interpolation=inter)

    return resized

ModuleNotFoundError: No module named 'cv2'

# 6.2 - Image Arithmetic

## Why is it important?

It's important because there will be cases where you will need to manipulate your images, and you're only limited to values between 0 and 255 for RGB.

The two main methods people take are:
- Wrapping around (such as NumPy)
- Clipping off excess values (such as OpenCV)

## How do We Utilize `openCV`s Clipping Tools?

`opencv.add()` and `opencv.subtract()` will take care of the clipping for us when we pass in NumPy arrays

Just make sure you use the `np.ones(image.shape, dtype = "uint8")` with type `uint8` in order to restrict values to 0 to 255

# 6.3 - The Four Bitwise Operators

* `AND`
* `OR`
* `XOR`
* `NOT`

## The Idea

A pixel is turned "off" if it has a value of zero and turned "on" if it has a value of one

## The Theory

1. `AND` - A bitwise AND is true if and only if both pixels are greater than zero.
2. `OR` - A bitwise OR is true if either of the two pixels are greater than zero
3. `XOR` - A bitwise XOR is true if and only if either of the two pixels are greater than zero, but not both.
4. `NOT` - A bitwise NOT inverts the “on” and “off” pixels in an image.

# 6.4 - Masking

## Why Do We Need Masking?

It helps us to only focus on the part of the image that interests us.

So in the case of a beach, we may only want to look at the palm trees. So what we can do is create a black canvas of the same size as the beach image, create a white (has RGB > 0) rectangle over the area we want and apply a bitwise `AND` operator on it

## How To Do It

1. Read in your image
2. Find the center of your image
3. Basically instead of drawing a rectange, you define your width and height and add half of each of those to the center. So for example:
4. Call the `cv2.bitwise_and` operator

`cv2.rectangle(mask, (cX - 75, cY - 75), (cX + 75 , cY + 75), 255, -1)`

# 6.5 - Splitting and Merging Channels

## What Are Channels?

So we know that color images consist of `Red`, `Green`, and `Blue`

## Let's Discuss in Context of Waves

So the channels of the wave would be as follows:
- the red channel would be very heavy since it's not represented
- the blue channel would be very light since it's represented a lot
- the green channel would be in the middle since it's slightly represented

## So.... How Do We Use It?

Remember that OpenCV stores RGB values for images using `[B, G, R]` instead of `[R, G, B]`

1. Split the image into the channels using `cv2.split(image)`
2. Display each channel individually using `cv2.imshow("Red", R)` for example
3. Merge the channels by using `cv2.merge([B, G, R])`

If you want to actually show the color in each channel, just pass in zeros for the other arguments in R, G, B format.

So for example, if you wanted to show red, you would have `[zeros, zeros, R]` noting that images in OpenCV again are stored in `[B, G, R]` instead of `[R, G, B]`

## Changing Color Spaces

Let's be real. We're not always going to have `RGB` color spaces. Sometimes, we'll have `HSV` color spaces as well

How do we convert between them?

### How to Convert

1. Import your image
2. Call the respective function, for example:

`
gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
`

This will call the `cvtColor` function on the image and pass in the `COLOR_BGR2GRAY` constant to tell `cv2` to conver this image to grayscale.

The other common ones are `COLOR_BGR2HSV` (HSV format) and `COLOR_BGR2LAB` (`L*a*b` format)